In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import collections as col
import os

In [2]:
path_to_data = "C:\\Users\\toby\\Dropbox\\NSF_SBIR_Grant\\SpectroElectroChem\\SHGC_VLT_master.xlsx"

In [3]:
# import the data as a panda dataframe
df = pd.read_excel(path_to_data,sheetname="Optical", header=1)

In [4]:
# preview the data to make sure that it looks okay
df.head(20)

,Device Name,Subfolder,Link,Voltage,State,'SHGC' (%T),%VLT,%T (600nm),Min wav,Max wav,"Incident Power (in range, W/m2)","Transmitted Power (in range, W/m2)",Transmitted Fraction (in range),"Incident Power (in range, W/m2).1","Transmitted Power (in range, W/m2).1",%VLT in range,deltaVLT,Unnamed: 17,Unnamed: 18
0,Sage_2012,NaN,http://www.sciencedirect.com/science/article/p...,NaN,Color,0.01503658,0.02575644,NaN,309.9850,2500.000000,992.412065,15.036584,0.015152,423.184464,13.753941,0.032501,0.549291,NaN,NaN
1,Sage_2012,NaN,NaN,NaN,Bleach,0.4440857,0.5750476,NaN,307.2552,2306.341958,985.769436,444.085721,0.450497,495.415341,307.075394,0.619834,0.549291,NaN,0.336965
2,Sage_2012,NaN,NaN,NaN,Difference,0.4290491,0.5492911,NaN,NaN,NaN,NaN,NaN,0.435345,NaN,NaN,0.587333,0.549291,NaN,307.000000
3,Reynolds_MCCP&ECP-Magenta_2012,NaN,http://pubs.rsc.org/en/Content/ArticleLanding/...,NaN,Color,0.286081,0.4912639,NaN,354.9600,795.040000,567.432330,286.081024,0.504168,533.730924,262.334903,0.491512,0.237340,NaN,NaN
4,Reynolds_MCCP&ECP-Magenta_2012,NaN,NaN,NaN,Bleach,0.4151851,0.7286039,NaN,354.9600,800.000000,575.345435,415.185136,0.721628,536.978073,389.074499,0.724563,0.237340,NaN,NaN
5,Reynolds_MCCP&ECP-Magenta_2012,NaN,NaN,NaN,Difference,0.1291041,0.2373401,NaN,NaN,NaN,NaN,NaN,0.217460,NaN,NaN,0.233052,0.237340,NaN,NaN
6,NexTint_ECD1,NaN,NaN,NaN,Color,0.2173684,0.229349,NaN,362.1800,1000.000000,719.814775,217.368360,0.301978,534.107783,122.472385,0.229303,0.033138,NaN,NaN
7,NexTint_ECD1,NaN,NaN,NaN,Bleach,0.1979798,0.2624866,NaN,362.4000,1000.000000,719.814775,197.979826,0.275043,534.107783,140.167838,0.262434,0.033138,NaN,NaN
8,NexTint_ECD1,NaN,NaN,NaN,Difference,-0.01938853,0.03313755,NaN,NaN,NaN,NaN,NaN,-0.026935,NaN,NaN,0.033131,0.033138,NaN,NaN
9,20140704_NexTint_P3HT-ITO,NaN,NaN,NaN,Color,0.4135625,0.4613483,NaN,348.6500,1000.000000,726.913785,413.562483,0.568929,534.107783,246.359985,0.461255,0.103395,NaN,NaN


In [5]:
# A command to easily see which devices spent the most time on the tester
namelives = df['Device Name'].value_counts(sort = False)

In [6]:
dnames = df['Device Name'].tolist()


In [7]:

anode = [] 
electrolyte = []
cathode = []
dates = []
# this loop splits the device name and extracts the date, anode, and cathode
for name in dnames:
    name.replace(" ","")
    if name.count('_') >= 3: 
        bins = name.split('_');
        dates.append(bins[0])
        if bins[1] == "ito":
            cathode.append(bins[2])
            electrolyte.append(bins[3])
            try: 
                anode.append(bins[4])
            except IndexError:
                anode.append("ito")
        else:
            cathode.append(bins[1])
            electrolyte.append(bins[2])
            anode.append(bins[3])
    else:
            dates.append('BadDate')
            cathode.append('BadCathode')
            electrolyte.append('BadElectrolyte')
            anode.append('BadAnode')
    

In [8]:
df['date']= pd.Series(dates, index=df.index)
df['cathode'] = pd.Series(cathode, index=df.index)
df['electrolyte'] = pd.Series(electrolyte, index=df.index)
df['anode'] = pd.Series(anode, index=df.index)

In [9]:
matdf = pd.concat([df['Device Name'], df['date'], df['cathode'], df['electrolyte'], df['anode']], 
                  axis = 1, keys = ['DeviceName',
                  'date','cathode','electrolyte','anode']);
matdf = matdf.sort_index(by='DeviceName', ascending = True);
matdf = matdf.drop_duplicates(subset='DeviceName');


In [10]:
counts = col.Counter(dnames)
counts = col.OrderedDict(sorted(counts.items()))
matdf['weeks alive'] = pd.Series([x / 3 for x in counts.values()], index=matdf.index)

In [11]:
dcathode = pd.concat([ matdf['cathode'], matdf['weeks alive']],
                     axis = 1, keys = [ 'cathode', 'weeks alive'], )
dcathode = dcathode[dcathode.cathode != 'liclo4']

In [12]:
dcathode.boxplot(column='weeks alive',by='cathode', rot='90')
plt.show()

In [13]:
import os
os.getcwd()

'C:\\Users\\toby\\Documents\\pyladies_intro_to_data_science-master\\pyladies_intro_to_data_science-master'